In [31]:
import pandas as pd
import numpy as np
import copy as copy
from datetime import datetime

In [32]:
case_years = [
    "./precog_data/csv/cases/cases_2010.csv",
    "./precog_data/csv/cases/cases_2011.csv", 
    "./precog_data/csv/cases/cases_2012.csv", 
    "./precog_data/csv/cases/cases_2013.csv", 
    "./precog_data/csv/cases/cases_2014.csv", 
    "./precog_data/csv/cases/cases_2015.csv", 
    "./precog_data/csv/cases/cases_2016.csv", 
    "./precog_data/csv/cases/cases_2017.csv", 
    "./precog_data/csv/cases/cases_2018.csv", ]

In [33]:
judge_df = pd.read_csv("./precog_data/csv/judges_clean.csv")
judge_df = judge_df.drop(["state_code", "dist_code", "court_no", "judge_position"], axis = 1)
judge_df = judge_df.dropna()

judge_merge_df = pd.read_csv("./precog_data/csv/keys/judge_case_merge_key.csv")
judge_merge_df = judge_merge_df.drop(["ddl_filing_judge_id"], axis = 1)
judge_merge_df = judge_merge_df.dropna()

In [34]:
judge_merge_df = pd.read_csv("./precog_data/csv/keys/judge_case_merge_key.csv")
judge_merge_df = judge_merge_df.drop(["ddl_filing_judge_id"], axis = 1)
judge_merge_df = judge_merge_df.dropna()

In [35]:
merged_judge = pd.merge(
    left = judge_df,
    right = judge_merge_df,
    left_on = "ddl_judge_id",
    right_on = "ddl_decision_judge_id",
    how="inner"
)

merged_judge = merged_judge.drop(["ddl_judge_id", "ddl_decision_judge_id"], axis = 1)

In [36]:
case_year_2010 = case_years[0]
year_df = pd.read_csv(case_year_2010, usecols = ["ddl_case_id","female_defendant", "female_petitioner","disp_name","date_of_filing","date_first_list", "state_code"])
year_df.dropna()

year_df = year_df[(year_df["female_defendant"] == "0 male") | (year_df["female_defendant"] == "1 female")]
year_df = year_df[(year_df["female_petitioner"] == "0 male") | (year_df["female_petitioner"] == "1 female")]


temp_df = pd.merge(
    left = merged_judge,
    right = year_df,
    on = "ddl_case_id",
    how="inner"
)
accumulator_df = temp_df

In [38]:
for i in range(1,9):
    
    year_df = pd.read_csv(case_years[i], usecols = ["ddl_case_id","female_defendant", "female_petitioner","disp_name","date_of_filing","date_first_list", "state_code"])
    year_df.dropna()
    
    year_df = year_df[(year_df["female_defendant"] == "0 male") | (year_df["female_defendant"] == "1 female")]
    year_df = year_df[(year_df["female_petitioner"] == "0 male") | (year_df["female_petitioner"] == "1 female")]
    
    temp_df = pd.merge(
        left = merged_judge,
        right = year_df,
        on = "ddl_case_id",
        how="inner"
    )
    
    accumulator_df = pd.concat([accumulator_df, temp_df], axis = 0)
    save_df1 = copy.deepcopy(accumulator_df)

In [39]:
act_df = pd.read_csv("./precog_data/csv/acts_sections.csv")
accumulator_df = pd.merge(
    left = accumulator_df,
    right = act_df,
    on = "ddl_case_id",
    how = "inner"
)


/var/folders/dn/j6pxg4n515q0bqtnmk38b6g00000gn/T/ipykernel_60363/1642017826.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  act_df = pd.read_csv("./precog_data/csv/acts_sections.csv")


In [40]:
save_df2 = copy.deepcopy(accumulator_df)

In [ ]:
accumulator_df = copy.deepcopy()

In [63]:
accumulator_df = accumulator_df.drop(["bailable_ipc", "number_sections_ipc", "section", "act"], axis = 1)
accumulator_df = accumulator_df[accumulator_df.notnull()]

In [64]:
act_df = pd.read_csv("./precog_data/csv/keys/disp_name_key.csv")
accumulator_df = pd.merge(
    left = accumulator_df,
    right = act_df,
    on = "disp_name",
    how = "inner"
)

In [65]:
accumulator_df = accumulator_df.drop(["count"], axis = 1)
accumulator_df = accumulator_df.dropna()

In [66]:
def calculate_days(a, b):
    # distance between the smaller a, and the bigger b date
    
    date_object_2 = datetime.strptime(str(a), '%Y-%m-%d').date()
    date_object_1 = datetime.strptime(str(b), '%Y-%m-%d').date()
        
    return int((date_object_1 - date_object_2).days)

In [67]:
def calculate_days_format(a, b):
    # distance between the smaller a, and the bigger b date
    
    date_object_2 = datetime.strptime(str(a), '%d-%m-%Y').date()
    date_object_1 = datetime.strptime(str(b), '%Y-%m-%d').date()
        
    return int((date_object_1 - date_object_2).days)

In [68]:
accumulator_df["filing_after_2010"] = accumulator_df.apply(lambda row: calculate_days("2010-01-01", row["date_of_filing"]), axis = 1)

In [69]:
accumulator_df["first_list_after_2010"] = accumulator_df.apply(lambda row: calculate_days("2010-01-01", row["date_first_list"]), axis = 1)

In [70]:
accumulator_df["judge_exp_first"] = accumulator_df.apply(lambda row: calculate_days_format(row["start_date"], row["date_first_list"]), axis = 1)

In [71]:
accumulator_df = accumulator_df[accumulator_df["judge_exp_first"] > 0]
accumulator_df = accumulator_df[accumulator_df["filing_after_2010"] > 0]
accumulator_df = accumulator_df[accumulator_df["first_list_after_2010"] > 0]
accumulator_df = accumulator_df[(accumulator_df["first_list_after_2010"] - accumulator_df["filing_after_2010"]) > 0]

In [72]:
accumulator_df = accumulator_df[accumulator_df["disp_name_s"] != "disposition var missing"]

In [ ]:
accumulator_df = accumulator_df.drop(["start_date","ddl_case_id", "end_date" , "date_of_filing", "date_first_list", "disp_name_s"], axis = 1)

### NOW WE HAVE OUR CLEANED DATASET:

In [116]:
accumulator_df.to_csv("./precog_data/dataset.csv")